# Transpose Demo

#### Today's Topics
1. Transactions and Blocks
2. NFTs
3. Fungible Tokens
4. Accounts
5. What Next?

## Transactions and Blocks

In [3]:
import os
from transpose import Transpose
from dotenv import load_dotenv
load_dotenv()

# create a new API client with an API key
api = Transpose(os.environ.get('API_KEY'))

AttributeError: 'Transpose' object has no attribute 'api_key'